In [1]:
from tika import parser   # Importieren des Moduls parser von tika um den Inhalt des PDFs auszulesen
import re # Importieren des Regexmoduls
from mailmerge import MailMerge # Importieren von MailMerge aus docx-mailmerge2
from datetime import datetime # Importieren von datetime

In [2]:
parsed_pdf = parser.from_file('20220826_Mahnung.pdf')
# Einlesen des PDFs; diese Datei liegt im gleichen Ordner wie das Jupyter Notebook
  
data = parsed_pdf['content']   # Zuweisung des Inhalts (Content) des eingelesenen PDFs an die Variable data
  
print(data) # Anzeige des ausgelesenen Contents, d.h. der Variablen data
  
print(type(data)) # Anzeige des Dateityps der Variablen data









































Hochschule XYX    Telefon: 
Bibliothek Email:

Homepage:

Adresse Bibliothek

Herr/Frau
Max Mustermann
Straße 1

PLZ1 Ort1

Datum: 26.08.2022
   Benutzernummer: 979xxxxxxxxxxx

1. Aufforderung zur Medienrückgabe und Kostenrechnung

Wir bitten um SOFORTIGE RÜCKGABE der unten aufgeführten Medien,
deren Leihfrist abgelaufen ist.

 [Verfasser] / Titel
Mediennummer / Signatur
Leihfristende: 
Entliehen in Zweigstelle: xxxxx

Mahngebühr: EUR xxx

Bibliothek XY Tel:
Straße Fax: 
Ort



Hochschule XYX    Telefon: 
Bibliothek Email:

Homepage:

Adresse Bibliothek

Herr/Frau
Maxi Musterfrau
Straße 2

PLZ2 Ort2

Datum: 26.08.2022
   Benutzernummer: 979xxxxxxxxxxx

1. Aufforderung zur Medienrückgabe und Kostenrechnung

Wir bitten um SOFORTIGE RÜCKGABE der unten aufgeführten Medien,
deren Leihfrist abgelaufen ist.

 [Verfasser] / Titel
Mediennummer / Signatur
Leihfristende: 
Entliehen in Zweigstelle: xxxxx

Mahngebühr: EUR xxx

Bibliothek XY Tel:
Straße Fax: 


In [3]:
input_address = re.findall(r'Herr/Frau.*\n.*\n.*\n.*\n.*', data) # Verwenden der Funktion findall des Regexmoduls
# um jedes Auftreten der regular expression in data zu finden; Ergebnis ist eine Liste mit Strings
print(input_address) # Anzeige der gefundenen Daten

['Herr/Frau\nMax Mustermann\nStraße 1\n\nPLZ1 Ort1', 'Herr/Frau\nMaxi Musterfrau\nStraße 2\n\nPLZ2 Ort2', 'Herr/Frau\nMiniMax Mustekind\nStraße 3\n\nPLZ3 Ort3']


In [4]:
input_data = [] # Erzeugen einer leeren Liste 
for item in input_address:  # item ist Element der Liste, d.h. ein String
    item_export = {} # Erstellen eines leeren Dictionaries
    item_export['Name'] = (re.sub(r'Herr/Frau.*\n','',re.search(r'Herr/Frau.*\n.*', item).group(0))).rstrip() 
    # Ausschneiden des Names ohne Anrede Herr/Frau
    item_export['Adresszeile 1'] = re.sub(r'\n\n.*','',re.sub(r'Herr/Frau.*\n.*\n','',item))
    # Ausschneiden der Straße und Hausnummer 
    item_export['Stadt'] = (re.sub(r'Herr/Frau\n.*\n.*\n\n','',item)).rstrip()
    # Ausschneiden von PLZ und Ort
    input_data.append(item_export) # fügt das Dictionary zur Liste hinzu
print(input_data) # Anzeige der mit Dictionaries gefüllten Liste

[{'Name': 'Max Mustermann', 'Adresszeile 1': 'Straße 1', 'Stadt': 'PLZ1 Ort1'}, {'Name': 'Maxi Musterfrau', 'Adresszeile 1': 'Straße 2', 'Stadt': 'PLZ2 Ort2'}, {'Name': 'MiniMax Mustekind', 'Adresszeile 1': 'Straße 3', 'Stadt': 'PLZ3 Ort3'}]


In [5]:
template = 'Mahnbescheid_anonym.docx'
# template wird der String mit dem Dateinamen der Vorlage des Mahnbescheids mit Serienbrieffeldern zugewiesen;
# diese Datei liegt im gleichen Ordner wie das Jupyter Notebook

In [6]:
document = MailMerge(template, remove_empty_tables=True, auto_update_fields_on_open="always") 
# Einlesen der Vorlage durch MailMerge
print(document.get_merge_fields()) # Anzeige der Serienbrieffelder

{'Name', 'Adresszeile 1', 'Stadt'}


In [7]:
current_day = datetime.now().strftime('%Y%m%d') 
# Speichern des aktuellen Datums unter der Variablen current_day

In [8]:
document.merge_templates(input_data, separator='page_break') 
# Ausfüllen der Serienbrieffelder in der Vorlage durch die in der Liste
# der Dictionaries input_date gespeicherten Daten
document.write('output_mahnung'+current_day+'.docx') 
# Schreiben und Speichern des ausgefüllten Serienbriefes unter neuem Dateinamen

In [9]:
! libreoffice --headless --convert-to pdf output_mahnung$(date +"%Y%m%d").docx 
# Konvertieren des Word-Dokumentes zu PDF (als Bashbefehl durch !)
# folgende Warnung scheint keinen Einfluss zu haben: 
# Warning faild to launch javaldx - java may not function correctly

convert /home/katrin/2021-11-29-DL_Katrin_Gaertner/Modul_6/output_mahnung20220911.docx -> /home/katrin/2021-11-29-DL_Katrin_Gaertner/Modul_6/output_mahnung20220911.pdf using filter : writer_pdf_Export
Overwriting: /home/katrin/2021-11-29-DL_Katrin_Gaertner/Modul_6/output_mahnung20220911.pdf
